Draft 1

In [1]:
#XGB / FRACTALS Imports
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
import xgboost as xgb
import pickle as pickle
import os.path
import cv2
from skimage import io
import datetime as dt

import scipy
from sklearn.metrics import fbeta_score

from PIL import Image, ImageStat
print("------Imports complete-----")

/home/alex/NeuralNetworks/keras_tf/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


------Imports complete-----


In [2]:
#Fan Fei / KERAS Imports

# Tips on using pre-trained model in Keras
# http://stackoverflow.com/questions/41764041/fine-tuning-pretrained-model-in-keras/41791568
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# Furter import to allow for re-training / fine-tuning
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [3]:
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

train_path = "/media/alex/B44254FE4254C730/Users/Alex/Downloads/Kaggle Data/train-jpg/train-jpg/"
test_path = "/media/alex/B44254FE4254C730/Users/Alex/Downloads/Kaggle Data/test-jpg/"

train = pd.read_csv("/home/alex/Desktop/Rainforest/Data/train_v2.csv")
test =  pd.read_csv("/home/alex/Desktop/Rainforest/Data/sample_submission_v2.csv")
#Location for saving serialized objects
obj_save_path = "/home/alex/Desktop/Rainforest/Models/XGB/Objects/"
subm_output_path = "/home/alex/Desktop/Rainforest/Submissions/"
print("------Paths Built-----")

------Paths Built-----


# Define Extraction Methods

### Method 1: Predefined Features

In [82]:
def extract_features_PDF(df, data_path, name):
    #Wrapper method to check for existence of extracted features.
    
    if(os.path.isfile(obj_save_path + 'PDF_Feature_Extraction_' + name + '.p')):
        print('---- ' + 'Previous extraction found:' +'PDF_Feature_Extraction_' + name + '.p' +' ----' )
        im_features = pickle.load(open(obj_save_path + 'PDF_Feature_Extraction_' + name + '.p','rb'))
    else:
        print('---- ' + 'Previous extraction NOT found:' +'PDF_Feature_Extraction_' + name + '.p' +' ----' )
        print('---- ' + 'Extracting features' + ' ----')
        im_features = _extract_featuresPDF(df, data_path, name)
        pickle.dump(im_features, open(obj_save_path + 'PDF_Feature_Extraction_' + name + '.p','wb'))
    
    return im_features
        
        


def _extract_features_PDF(df, data_path, name):
	#create a copy of original panda dataframe so that extracted features can be added.
	im_features = df.copy()
	r_mean = []
	g_mean = []
	b_mean = []

	r_std=[]
	g_std=[]
	b_std=[]

	r_max=[]
	g_max=[]
	b_max=[]
	
	r_min=[]
	g_min=[]
	b_min=[]

	r_kurtosis=[]
	g_kurtosis=[]
	b_kurtosis=[]

	r_skewness=[]
	g_skewness=[]
	b_skewness=[]

	for image_name in tqdm(im_features.image_name.values, miniters=1000):
		
		if "file_" in image_name:
			continue 
		im = Image.open(data_path + image_name + '.jpg')
		#creating 256x256x3 array
		im= np.array(im)[:,:,:3]
		
		# im[:,:,0] has dimension 256x256; .ravel() flattens into 256^2 x 1
		r = im[:,:,0].ravel()
		g = im[:,:,1].ravel()
		b = im[:,:,2].ravel()
		
		r_mean.append(np.mean(r))
		g_mean.append(np.mean(g))
		b_mean.append(np.mean(b))

		r_std.append(np.std(r))
		g_std.append(np.std(g))
		b_std.append(np.std(b))

		r_max.append(np.max(r))
		g_max.append(np.max(g))
		b_max.append(np.max(b))
		
		r_min.append(np.min(r))
		g_min.append(np.min(g))
		b_min.append(np.min(b))
		
		r_kurtosis.append(scipy.stats.kurtosis(r))
		g_kurtosis.append(scipy.stats.kurtosis(g))
		b_kurtosis.append(scipy.stats.kurtosis(b))
		
		r_skewness.append(scipy.stats.skew(r))
		g_skewness.append(scipy.stats.skew(g))
		b_skewness.append(scipy.stats.skew(b))
        
        

	#Add extracted features to pandas frame
	#print(im_features['r_mean'].shape)
	#print(r_mean.shape)
	im_features['r_mean'] = r_mean
	im_features['g_mean'] = g_mean
	im_features['b_mean'] = b_mean
	
	im_features['r_std'] = r_std
	im_features['g_std'] = g_std
	im_features['b_std'] = b_std
	
	im_features['r_max'] = r_max
	im_features['g_max'] = g_max
	im_features['b_max'] = b_max
	
	im_features['r_min'] = r_min
	im_features['g_min'] = g_min
	im_features['b_min'] = b_min
	
	im_features['r_kurtosis'] = r_kurtosis
	im_features['g_kurtosis'] = g_kurtosis
	im_features['b_kurtosis'] = b_kurtosis
	
	im_features['r_skewness'] = r_skewness
	im_features['g_skewness'] = g_skewness
	im_features['b_skewness'] = b_skewness
	
	#return a pandas dataframe with above features extracted
	return im_features

### Method 2: ResNet50 with top layer removed

In [5]:

def extract_features_RN50(df, data_path, name):
    im_features = df.copy()
    base_model = ResNet50(weights = 'imagenet', include_top = False)
    
    feats_all = None
    #Attempt to resume
    if(os.path.isfile(obj_save_path + 'RN50_feature_Extraction_' + name + '.p')):
        print('----Previous Extraction found----')
        file = open(obj_save_path + 'RN50_feature_Extraction_' + name + '.p','rb')
        feats_all = pickle.load(file)
        feats_all = np.array(feats_all)
        file.close()
        im_features = im_features[feats_all.shape[0]:]
        print('----resuming at ' + str(feats_all.shape[0]) + ' ------')
    
        
    
    
    i = 1
    for image_name in tqdm(im_features.image_name.values, miniters=1000):
        im = img_to_array(load_img(data_path + image_name + '.jpg', target_size=[224,224]))
        im = np.expand_dims(im, axis=0)
        feats = base_model.predict(im)
        tmp = np.array(feats.ravel())
        
        if feats_all is None:
            feats_all = tmp
            
        else:
            feats_all = np.vstack((feats_all,tmp))
            
        if (i % 500) == 0:
            print('-----Checkpoint at ' + str(i) + ' ------')
            im_features = pd.DataFrame(feats_all)
            pickle.dump(im_features, open(obj_save_path + 'RN50_feature_Extraction_' + name + '.p','wb'))
            
        i = i+1
    
    im_features = pd.DataFrame(feats_all)
    pickle.dump(im_features, open(obj_save_path + 'RN50_feature_Extraction_' + name + '.p','wb'))
    print('----' + name + ' Extraction complete--------')
    return im_features






## Define Learning Methods:

### XGB

In [11]:
#XGBoost:
#Will add parameters to allow for tuning
def _XGBoost(_X_train, _X_test, _y, _name):
    y_pred_test = np.zeros((_X_test.shape[0], n_classes))
    y_pred_train = np.zeros((_X_train.shape[0], n_classes))
    
    if os.path.isfile(obj_save_path + _name + '.p') and os.path.isfile(obj_save_path + _name +'train' + '.p'):
        print('----' + _name + ' found ----')
        y_pred_test = pickle.load(open(obj_save_path + _name + '.p','rb'))
        y_pred_train = pickle.load(open(obj_save_path + _name + 'train'+ '.p','rb'))



    else:
        print('----' + _name + 'not found: fitting XGB Model ----')
        for class_i in tqdm( range(n_classes), miniters = 1 ):
            model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, \
                                      silent=True, objective='binary:logistic', nthread=-1, \
                                      gamma=0, min_child_weight=1, max_delta_step=0, \
                                      subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                                      reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                                      base_score=0.5, seed=random_seed, missing=None)
            model.fit(_X_train, _y[:, class_i])
            #Apply model i
            y_pred_test[:,class_i] = model.predict_proba(_X_test)[:,1]
            y_pred_train[:,class_i] = model.predict_proba(_X_train)[:,1]
    
        pickle.dump(y_pred_test, open(obj_save_path + _name + '.p','wb'))
        pickle.dump(y_pred_train, open(obj_save_path + _name +'train'+ '.p','wb'))
    return list((y_pred_test, y_pred_train))

## Apply Models

### ResNet50 (-1 layer) Model:

In [65]:
#Extract Features
train_features = extract_features_RN50(train,train_path, 'train')

test_features = extract_features_RN50(test,test_path,'test')


----Previous Extraction found----


0it [00:00, ?it/s]

----resuming at 40479 ------


----train Extraction complete--------
----Previous Extraction found----


0it [00:00, ?it/s]

----resuming at 40669 ------


----test Extraction complete--------


In [31]:
#Prepare for XGBoost:
X = np.array(train_features)
X_test  = np.array(test_features)
y = train['tags'].str.get_dummies(sep=' ')
y = np.array(y, np.uint8)
n_classes = y.shape[1]

flatten = lambda l: [item for sublist in l for item in sublist]

tmp = []
for l in train['tags'].values:
    tmp.append(l.split(' '))
    
labels = np.array(list(set(flatten(tmp))))


print('----Check shapes----')
print('X: ' + str(X.shape))
print('y: ' + str(y.shape))
print('X_test: ' + str(X_test.shape))
print('Classes: ' + str(n_classes))

#Apply XGBoost:
XGB_Ret = _XGBoost(X,X_test,y, 'RN50_XGB_Pred') 
y_pred_test = XGB_Ret[0]
y_pred_train = XGB_Ret[1]




----Check shapes----
X: (40479, 2048)
y: (40479, 17)
X_test: (40669, 2048)
Classes: 17
----RN50_XGB_Pred found ----


In [121]:
#Define classification thresholds as column averages
class_thresholds_train = np.zeros(17)
class_thresholds_test = np.zeros(17)

for col in range(y_pred_test.shape[1]):
    class_thresholds_test[col] = np.average(y_pred_test[:,col])

for col in range(y_pred_train.shape[1]):
    class_thresholds_train[col] = np.average(y_pred_train[:,col])
    

#apply thresholds:
y_pred_c_test = (y_pred_test > class_thresholds_test)*1.0
y_pred_c_train = (y_pred_train > class_thresholds_train)*1.0

#y_pred_c_test = (y_pred_test > 0.22222)*1.0
#y_pred_c_train = (y_pred_train > 0.22222)*1.0

#Return train score
print('f_beta score:')
print(fbeta_score(y,y_pred_c_train,beta=2, average='samples'))

#Build submission
labels = train['tags'].str.get_dummies(sep=' ').columns
test_labels = []
for row in range(y_pred_c_test.shape[0]):
    test_labels.append(' '.join(labels[y_pred_c_test[row,:]==1]))
Submission_RN50 = test.copy()
Submission_RN50.drop('tags', axis = 1)
Submission_RN50['tags'] = test_labels
Submission_RN50.to_csv(str(subm_output_path + 'submission_RN50.csv'), index = False)
print('---- End ResNet50 Model----')
y = train['tags'].str.get_dummies(sep=' ')


    

f_beta score:
0.819351442538
---- End ResNet50 Model----


## Pre-defined feature Model:

In [83]:
#Pre-defined feature model = PDFModel
PDFModel_train_features = extract_features_PDF(train, train_path, 'train')
PDFModel_test_features = extract_features_PDF(test, test_path, 'test')


---- Previous extraction found:PDF_Feature_Extraction_train.p ----
---- Previous extraction found:PDF_Feature_Extraction_test.p ----


In [100]:
#Prepare for XGBoost:
X = np.array(PDFModel_train_features.drop(['image_name', 'tags'],axis = 1))
X_test  = np.array(PDFModel_test_features.drop(['image_name', 'tags'],axis = 1))
y = train['tags'].str.get_dummies(sep=' ')

y = np.array(y, np.uint8)
n_classes = y.shape[1]

flatten = lambda l: [item for sublist in l for item in sublist]

tmp = []
for l in train['tags'].values:
    tmp.append(l.split(' '))
    
labels = np.array(list(set(flatten(tmp))))

print('----Check shapes----')
print('X: ' + str(X.shape))
print('y: ' + str(y.shape))
print('X_test: ' + str(X_test.shape))
print('Classes: ' + str(n_classes))

#Apply XGBoost:
XGB_Ret = _XGBoost(X,X_test,y, 'PDFModel_XGB_Pred') 
y_pred_test = XGB_Ret[0]
y_pred_train = XGB_Ret[1]




  0%|          | 0/17 [00:00<?, ?it/s]

----Check shapes----
X: (40479, 18)
y: (40479, 17)
X_test: (40669, 18)
Classes: 17
----PDFModel_XGB_Prednot found: fitting XGB Model ----


100%|██████████| 17/17 [00:38<00:00,  2.22s/it]


In [117]:
#Define classification thresholds as column averages
class_thresholds_train = np.zeros(17)
class_thresholds_test = np.zeros(17)

for col in range(y_pred_test.shape[1]):
    class_thresholds_test[col] = np.average(y_pred_test[:,col])

for col in range(y_pred_train.shape[1]):
    class_thresholds_train[col] = np.average(y_pred_train[:,col])

#apply thresholds:
#y_pred_c_test = (y_pred_test > class_thresholds_test)*1.0
#y_pred_c_train = (y_pred_train > class_thresholds_train)*1.0

y_pred_c_test = (y_pred_test > .2222222)*1.0
y_pred_c_train = (y_pred_train > .222222)*1.0


#Return train score
print('f_beta score:')
print(fbeta_score(y,y_pred_c_train,beta=2, average='samples'))

#Build submission
labels = train['tags'].str.get_dummies(sep=' ').columns
test_labels = []
for row in range(y_pred_c_test.shape[0]):
    test_labels.append(' '.join(labels[y_pred_c_test[row,:]==1]))
Submission_PDFModel = test.copy()
Submission_PDFModel.drop('tags', axis = 1)
Submission_PDFModel['tags'] = test_labels
Submission_PDFModel.to_csv(str(subm_output_path + 'submission_PDFModel.csv'), index = False)
print('---- End ResNet50 Model----')







f_beta score:
0.905139358748
---- End ResNet50 Model----


In [107]:
class_thresholds_train

array([ 0.30430032,  0.00842468,  0.02131553,  0.00825091,  0.00244761,
        0.70233841,  0.05164864,  0.00248333,  0.11067056,  0.09048466,
        0.06665388,  0.17947589,  0.92670635,  0.19945985,  0.00844782,
        0.00517558,  0.18303542])

In [109]:
np.average(class_thresholds_test)

0.16776805025407501

### Threshold Tuning?

In [116]:
t = np.linspace(0,1,num = 100)

for a in t:
    y_pred_c_train = (y_pred_train > a)*1.0
    print(a)
    print(fbeta_score(y,y_pred_c_train,beta=2, average='samples'))
    print('----')
    
    
    

0.0
0.4836858915
----
0.010101010101
0.740317510686
----
0.020202020202
0.789891231724
----
0.030303030303
0.819730410699
----
0.040404040404
0.835879451995
----
0.0505050505051
0.847709337374
----
0.0606060606061
0.856824235621
----
0.0707070707071
0.864028228468
----
0.0808080808081
0.870154321625
----
0.0909090909091
0.875287923792
----
0.10101010101
0.880139391838
----
0.111111111111
0.884156517544
----
0.121212121212
0.88781175259
----
0.131313131313
0.891144796393
----
0.141414141414
0.894142614139
----
0.151515151515
0.896493518004
----
0.161616161616
0.898336970631
----
0.171717171717
0.90020124565
----
0.181818181818
0.901987759249
----
0.191919191919
0.903262491892
----
0.20202020202
0.904156946317
----
0.212121212121
0.904887616763
----
0.222222222222
0.905139358748
----
0.232323232323
0.905073142298
----
0.242424242424
0.905070027506
----
0.252525252525
0.904636801782
----
0.262626262626
0.903814320046
----
0.272727272727
0.902935793507
----
0.282828282828
0.901829114711
--

/home/alex/NeuralNetworks/keras_tf/lib/python3.4/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.894195971651
----
0.343434343434
0.892121884461
----
0.353535353535
0.890332731516
----
0.363636363636
0.88844288647
----
0.373737373737
0.886095842565
----
0.383838383838
0.884125165324
----
0.393939393939
0.882319190919
----
0.40404040404
0.880081887039
----
0.414141414141
0.878152415357
----
0.424242424242
0.875820275184
----
0.434343434343
0.873468777831
----
0.444444444444
0.871073517376
----
0.454545454545
0.86870383496
----
0.464646464646
0.866066079789
----
0.474747474747
0.863446904088
----
0.484848484848
0.861017590234
----
0.494949494949
0.858271409092
----
0.505050505051
0.855672605492
----
0.515151515152
0.853063585479
----
0.525252525253
0.849939681127
----
0.535353535354
0.847235330727
----
0.545454545455
0.844113135272
----
0.555555555556
0.840995805985
----
0.565656565657
0.838010860741
----
0.575757575758
0.834985826571
----
0.585858585859
0.831585862331
----
0.59595959596
0.828094257815
----
0.606060606061
0.824799332671
----
0.616161616162
0.821320717972
----
0.62